In [41]:
%load_ext autoreload
%autoreload 2
from glob import glob
import os
from pathlib import Path
import re
import sys
import yaml
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from func.ml_utils import Classifier
from func.preprocessing import get_dummies

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET]

train_paths = glob('../feature/raw_main/*_train.gz')
test_paths = glob('../feature/raw_main/*_test.gz')

train_paths = [path for path in train_paths 
               if not path.count('V') 
               and not path.count('day_no') 
              ]
test_paths  = [path for path in test_paths  
               if not path.count('V') 
               and not path.count('day_no') 
              ]

train_df = parallel_load_data(train_paths)
test_df = parallel_load_data(test_paths)

In [212]:
filepath = '../output/0826_transaction_new_user.csv'
target_ids = pd.read_csv(filepath).values.ravel()
all_data = pd.concat([train_df, test_df], axis=0, ignore_index=False)

train_data = train_df[train_df[COLUMN_ID].isin(target_ids)]
# data = all_data[all_data[COLUMN_ID].isin(target_ids)]
data = all_data
data['is_train'] = ~data[COLUMN_TARGET].isnull()*1
print(data.shape)

(1097231, 57)


In [220]:
col_target = 'is_train'
COLUMNS_CATEGORY = get_categorical_features(data, COLUMNS_IGNORE)
df_ohe = get_dummies(data, COLUMNS_CATEGORY)
train = df_ohe.drop(col_target, axis=1)
Y = df_ohe[col_target]

In [228]:
from sklearn.model_selection import StratifiedKFold

start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())[:13]

n_splits = 5
kfold = StratifiedKFold(n_splits=n_splits)
kfold = list(kfold.split(train, Y))
params = {}
metric = 'auc'
model_type = 'lgb'
score_list = []
feim_list  = []
y_pred = np.zeros(len(train))
test_preds = []
use_cols = [col for col in df_ohe.columns if col not in COLUMNS_IGNORE+[col_target]
#             and col.startswith('C')
            and not col.startswith('D') 
#             and not col.startswith('C')
            and not col.startswith('addr')
#             and not col.startswith('card')
            and not col.startswith('dist')
#             and not col.startswith('M')
            and not col.startswith('R_email')
            and not col.startswith('P_email')
#             and not col.count('Amt')
#             and not col.count('ProductCD')
           ]
model_map = {}

for n_fold, (trn_idx, val_idx) in enumerate(kfold):
    x_train = train.iloc[trn_idx][use_cols]
    y_train = Y.iloc[trn_idx]
    x_valid = train.iloc[val_idx][use_cols]
    y_valid = Y.iloc[val_idx]

    score, oof_pred, test_pred, feim, model = Classifier(
        model_type=model_type,
        x_train=x_train,
        y_train=y_train,
        x_valid=x_valid,
        y_valid=y_valid,
        x_test=[],
        params=params,
        get_model=True,
    )
    score_list.append(score)
    y_pred[val_idx] = oof_pred
    test_preds.append(test_pred)
    model_map[n_fold] = model
    
    feim.rename(columns={'importance': f'importance_fold{n_fold+1}'}, inplace=True)
    feim.set_index('feature', inplace=True)
    feim_list.append(feim)
    
cv_score = np.mean(score_list)
feim_df = pd.concat(feim_list, axis=1)
feim_df['importance_avg'] = feim_df.mean(axis=1)
feim_df.sort_values(by='importance_avg', ascending=False, inplace=True)

Training until validation scores don't improve for 100 rounds.
[200]	valid_0's auc: 0.5642
[400]	valid_0's auc: 0.573208
[600]	valid_0's auc: 0.579275
[800]	valid_0's auc: 0.582753
[1000]	valid_0's auc: 0.58525
[1200]	valid_0's auc: 0.587605
[1400]	valid_0's auc: 0.589973
[1600]	valid_0's auc: 0.591295
[1800]	valid_0's auc: 0.592285
[2000]	valid_0's auc: 0.593495
[2200]	valid_0's auc: 0.594378
[2400]	valid_0's auc: 0.59491
[2600]	valid_0's auc: 0.596184
[2800]	valid_0's auc: 0.597243
[3000]	valid_0's auc: 0.59801
[3200]	valid_0's auc: 0.599095
[3400]	valid_0's auc: 0.599865


KeyboardInterrupt: 

In [165]:
from func.xray_wrapper import Xray_Cal
xray = Xray_Cal(ignore_list=COLUMNS_IGNORE)

xray_result = pd.DataFrame()
for fold_num, (trn_idx, val_idx) in zip(tqdm(range(n_splits)), list(kfold)):
    xray.model = model_map[fold_num]
    xray, tmp_result = xray.get_xray(base_xray=train[use_cols].iloc[trn_idx, :], fold_num=fold_num, parallel=False)
    tmp_result.rename(columns={'xray' : f'xray_{fold_num}'}, inplace=True)
    
    if len(xray_result):
        xray_result = xray_result.merge(tmp_result, how='inner', on=['N', 'feature', 'value'])
    else:
        xray_result = tmp_result
        
# X-Rayの平均を出力（主にこれを可視化する）        
xray_cols = [col for col in xray_result.columns if col.count('xray')]
xray_result['xray_avg'] = xray_result[xray_cols].mean(axis=1)

 20%|██        | 1/5 [01:59<07:59, 119.78s/it]

FOLD: 0


 40%|████      | 2/5 [04:02<06:01, 120.55s/it]

FOLD: 1


 60%|██████    | 3/5 [06:08<04:04, 122.20s/it]

FOLD: 2


 80%|████████  | 4/5 [08:40<02:11, 131.19s/it]

FOLD: 3


100%|██████████| 5/5 [10:45<00:00, 129.41s/it]

FOLD: 4


In [166]:
xray_result.to_csv('../output/0826_ieee__adversarial_clf_xray__high_fraud_user.csv', index=False)

In [202]:
col_target = COLUMN_TARGET
COLUMNS_CATEGORY = get_categorical_features(train_data, COLUMNS_IGNORE)
train_data['diff_email'] = (train_data['P_emaildomain'] != train_data['R_emaildomain'])*1
df_ohe = get_dummies(train_data, COLUMNS_CATEGORY)
train = df_ohe.drop(col_target, axis=1)
Y = df_ohe[col_target]

In [209]:
from sklearn.model_selection import StratifiedKFold

start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())[:13]

seed = 1208
params = {
    'num_threads': -1,
    'num_leaves': 31,
    'objective':'binary',
    "boosting": "gbdt",
    'max_depth': -1,
    "min_child_samples": 20,
    "bagging_freq": 2,
    "bagging_fraction": 0.9,
    "colsample_bytree": 0.3,
    "lambda_l1": 0.1,
    "lambda_l2": 1.0,
    "verbosity": -1,
    'random_seed': seed,
    'bagging_seed':seed,
    'feature_fraction_seed':seed,
    'data_random_seed':seed
}
n_splits = 5
kfold = StratifiedKFold(n_splits=n_splits)
kfold = list(kfold.split(train, Y))
metric = 'auc'
model_type = 'lgb'
score_list = []
feim_list  = []
y_pred = np.zeros(len(train))
test_preds = []

use_cols = [col for col in df_ohe.columns if col not in COLUMNS_IGNORE+[col_target]
            and not col.startswith('D') 
            and not col.startswith('addr')
            and not col.startswith('dist')
            and not col.startswith('M')
            and not col.count('ProductCD')
            and not col.startswith('card')
            and not col.startswith('C')
#             and not col.startswith('R_email')
            and not col.count('Amt')
           ]

model_map = {}

for n_fold, (trn_idx, val_idx) in enumerate(kfold):
    x_train = train.iloc[trn_idx][use_cols]
    y_train = Y.iloc[trn_idx]
    x_valid = train.iloc[val_idx][use_cols]
    y_valid = Y.iloc[val_idx]

    score, oof_pred, test_pred, feim, model = Classifier(
        model_type=model_type,
        x_train=x_train,
        y_train=y_train,
        x_valid=x_valid,
        y_valid=y_valid,
        x_test=[],
        params=params,
        get_model=True,
    )
    score_list.append(score)
    y_pred[val_idx] = oof_pred
    test_preds.append(test_pred)
    model_map[n_fold] = model
    
    feim.rename(columns={'importance': f'importance_fold{n_fold+1}'}, inplace=True)
    feim.set_index('feature', inplace=True)
    feim_list.append(feim)
    
cv_score = np.mean(score_list)
feim_df = pd.concat(feim_list, axis=1)
feim_df['importance_avg'] = feim_df.mean(axis=1)
feim_df.sort_values(by='importance_avg', ascending=False, inplace=True)

Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[2]	valid_0's auc: 0.630697
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[7]	valid_0's auc: 0.616756
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[11]	valid_0's auc: 0.61114
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[2]	valid_0's auc: 0.673664
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[21]	valid_0's auc: 0.683892


In [210]:
COLUMN_PRED = 'prediction'
train_data[COLUMN_PRED] = y_pred
# train_data.to_csv('../output/0826_ieee__clf_pred__high_fraud_user.csv', index=False)
# train_data.to_csv('../output/0826_ieee__clf_pred__high_fraud_user__no_C.csv', index=False)
train_data.to_csv('../output/0826_ieee__clf_pred__high_fraud_user__no_C_card_amt.csv', index=False)

In [211]:
feim_df

,importance_fold1,importance_fold2,importance_fold3,importance_fold4,importance_fold5,importance_avg
feature,,,,,,
diff_email,450.226013,847.802979,854.136963,328.868011,1106.977905,717.602356
hour,337.876923,750.405945,1007.667114,347.086761,1100.308472,708.669067
R_emaildomain_gmail.com_dummie,413.959015,272.083405,604.232666,298.169006,294.503693,376.589539
R_emaildomain_hotmail.com_dummie,147.942001,0.000000,277.967896,129.268005,227.216370,156.478851
P_emaildomain_hotmail.com_dummie,16.031200,209.996994,151.251907,17.289400,350.631897,149.040283
P_emaildomain_aol.com_dummie,46.923401,130.825500,262.311096,26.742001,146.018311,122.564064
time_zone_6.5-10.0_dummie,0.000000,0.000000,155.776154,0.000000,306.685791,92.492386
P_emaildomain_gmail.com_dummie,0.000000,70.030197,159.113113,0.000000,165.086121,78.845886
P_emaildomain_yahoo.com_dummie,0.000000,52.709789,177.882294,0.000000,93.480202,64.814461


In [147]:
from func.xray_wrapper import Xray_Cal
xray = Xray_Cal(ignore_list=COLUMNS_IGNORE)

xray_result = pd.DataFrame()
for fold_num, (trn_idx, val_idx) in zip(tqdm(range(n_splits)), list(kfold)):
    xray.model = model_map[fold_num]
    xray, tmp_result = xray.get_xray(base_xray=train[use_cols].iloc[trn_idx, :], fold_num=fold_num, parallel=False)
    tmp_result.rename(columns={'xray' : f'xray_{fold_num}'}, inplace=True)
    
    if len(xray_result):
        xray_result = xray_result.merge(tmp_result, how='inner', on=['N', 'feature', 'value'])
    else:
        xray_result = tmp_result
        
# X-Rayの平均を出力（主にこれを可視化する）        
xray_cols = [col for col in xray_result.columns if col.count('xray')]
xray_result['xray_avg'] = xray_result[xray_cols].mean(axis=1)

  0%|          | 0/5 [00:00<?, ?it/s]


#========================================================================
# CALCULATION PROGRESS... COLUMN: C12 | VALUE: 1.0 | X-RAY: 0.37197371072412166
#========================================================================

#========================================================================
# CALCULATION PROGRESS... COLUMN: C12 | VALUE: 2.0 | X-RAY: 0.3658998118937136
#========================================================================

#========================================================================
# CALCULATION PROGRESS... COLUMN: C12 | VALUE: 3.0 | X-RAY: 0.36717564752600157
#========================================================================

#========================================================================
# CALCULATION PROGRESS... COLUMN: C12 | VALUE: 14.0 | X-RAY: 0.4285264146531947
#========================================================================

#========================================================================
# CALCULAT

 20%|██        | 1/5 [01:57<07:50, 117.51s/it]


#========================================================================
# CALCULATION PROGRESS... COLUMN: card6_debit_dummie | VALUE: 0 | X-RAY: 0.37341419443495394
#========================================================================
FOLD: 0

#========================================================================
# CALCULATION PROGRESS... COLUMN: C12 | VALUE: 1.0 | X-RAY: 0.36774382651286097
#========================================================================

#========================================================================
# CALCULATION PROGRESS... COLUMN: C12 | VALUE: 2.0 | X-RAY: 0.36689421725979837
#========================================================================

#========================================================================
# CALCULATION PROGRESS... COLUMN: C12 | VALUE: 3.0 | X-RAY: 0.4079502483084606
#========================================================================

#==============================================================

 40%|████      | 2/5 [04:05<06:02, 120.70s/it]

...


#========================================================================
# CALCULATION PROGRESS... COLUMN: card6_debit_dummie | VALUE: 0 | X-RAY: 0.3732097033583192
#========================================================================
FOLD: 2

#========================================================================
# CALCULATION PROGRESS... COLUMN: C12 | VALUE: 1.0 | X-RAY: 0.3802928165927534
#========================================================================

#========================================================================
# CALCULATION PROGRESS... COLUMN: C12 | VALUE: 2.0 | X-RAY: 0.3659059345691559
#========================================================================

#========================================================================
# CALCULATION PROGRESS... COLUMN: C12 | VALUE: 3.0 | X-RAY: 0.3650359629094049
#========================================================================

#=================================================================

 80%|████████  | 4/5 [08:14<02:02, 122.18s/it]


#========================================================================
# CALCULATION PROGRESS... COLUMN: card6_debit_dummie | VALUE: 0 | X-RAY: 0.37387207829191815
#========================================================================
FOLD: 3

#========================================================================
# CALCULATION PROGRESS... COLUMN: C12 | VALUE: 1.0 | X-RAY: 0.36668523564211214
#========================================================================

#========================================================================
# CALCULATION PROGRESS... COLUMN: C12 | VALUE: 2.0 | X-RAY: 0.3716495691134845
#========================================================================

#========================================================================
# CALCULATION PROGRESS... COLUMN: C12 | VALUE: 3.0 | X-RAY: 0.396468356973555
#========================================================================

#================================================================

100%|██████████| 5/5 [10:17<00:00, 122.23s/it]


#========================================================================
# CALCULATION PROGRESS... COLUMN: card6_debit_dummie | VALUE: 0 | X-RAY: 0.37393988494650504
#========================================================================
FOLD: 4


In [156]:
xray_result.to_csv('../output/0826_ieee__clf_xray__high_fraud_user.csv', index=False)